In [6]:
import torch

In [7]:
torch.cuda.is_available()

True

In [8]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [9]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [10]:
df.sample(5)

,Question,Answer
34061,Whatcha doin?,"Papa would say, ""I'M MINDING MY OWN DAMN BUSIN..."
3952,What do you call a sick russian?,A sicka blyat
59884,What does Popeye use to toss his salad?,Olive Oil.
61682,Why does the murder no longer use axe?,Because he realized it can't wash away his sins
40403,What's common to the cockpit of a modern fight...,The heads-up display


In [11]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [12]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,"Well, I told him I wouldn't - uh - go there an...",What am I supposed to say?
1,I don't know. I haven't decided yet.,Who's your partner?
2,Why?,"I don't think you trust anybody, do you?"
3,Bet your sweet ass we are.,Joe.
4,But Elaine --,I don't want you to go anywhere until you have...


In [13]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296389 entries, 0 to 296388
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Question  296389 non-null  object
 1   Answer    296389 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Tokenization

In [14]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [15]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs

Batchwise input and data preparation

In [16]:
import itertools

PAD_token = 0  # used for padding short sentences
SOS_token = 1  # start of sentence token
EOS_token = 2  # End-of-sentence token

def indexesFromSentence(voc , sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l , fillvalue = PAD_token):
    return list(itertools.zip_longest(*l , fillvalue = fillvalue))

def binaryMatrix(l , value=PAD_token):
    m = []
    for i , seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# returns padded input sequence tensor and lengths 
def inputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar , lengths

# returns padded target sequence tensor, padding mask , and max target length
def outputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar , mask , max_target_len

#returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        print("Input sentence-",pair[0])
        print("Output sentece-",pair[1])
        print("-----------------------")
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [20]:
import random

# Create a vocabulary instance
voc = Voc('test')

# Create pairs from your existing dataframe and normalize them
print("Creating pairs from dataframe...")
pairs = []
for _, row in dfn.iterrows():
    question = normalizeString(row['Question'])
    answer = normalizeString(row['Answer'])
    pairs.append([question, answer])

# Build vocabulary from pairs
print("Building vocabulary...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

# Example for validation
print("\nTesting with a small batch...")
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nBatch information:")
print("Input shape:", input_variable.shape)
print("Lengths:", lengths)
print("Output shape:", target_variable.shape)

Creating pairs from dataframe...
Building vocabulary...

Testing with a small batch...
Input sentence- well that is alright . . . alright . good . you will graduate in another year is not that right ? you know . . . i never finished college . i was a good student but i never finished . of course there was a war then .
Output sentece- i am not hungry .
-----------------------
Input sentence- joe it is not love . it is like . it is real strong like . and i got your information . now get off my back !
Output sentece- i ve got twenty hours left . i could die in here . and you re falling in love .
-----------------------
Input sentence- do you want us to call the cops and have them give you the boots ?
Output sentece- come on you ! before we slap you down .
-----------------------
Input sentence- what is the difference between jesus and a picture of jesus ?
Output sentece- it only takes one nail to hang up the picture .
-----------------------
Input sentence- plan ? what are you talking abo

In [21]:
def filterPair(voc, pair):
	# Return True if all words in both sentences are in vocab
	return all(word in voc.word2index for word in pair[0].split()) and \
		   all(word in voc.word2index for word in pair[1].split())

# Get valid pairs
valid_pairs = [pair for pair in pairs if filterPair(voc, pair)]

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(valid_pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nInput variable:")
print("input shape:", input_variable.shape)
print("lengths:", lengths)
print("\nTarget variable:")
print("Output shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len:", max_target_len)

Input sentence- i i do not think you have herr director . not yet . i mean it is quite n of course i will show it to you immediately .
Output sentece- oh ? have i seen it ?
-----------------------
Input sentence- excuse me missus . no time for the old in out i ve just come to read the meter . slide of bird s nest with eggs .
Output sentece- good . what do you want ?
-----------------------
Input sentence- why cannot you use a the restroom at a beatles reunion concert ?
Output sentece- because there is no john .
-----------------------
Input sentence- what does the secret service say when donald trump gets shot at ?
Output sentece- donald ! duck !
-----------------------
Input sentence- why does sirius black get all the girls ?
Output sentece- because he is a real dawg .
-----------------------

Input variable:
input shape: torch.Size([30, 5])
lengths: tensor([30, 29, 14, 14, 10])

Target variable:
Output shape: torch.Size([8, 5])
mask shape: torch.Size([8, 5])
max_target_len: 8
